In [9]:
import bqplot
import ytree
import yt
from collections import defaultdict
import ipywidgets as widgets
from traitlets import Unicode, validate, default, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw)

In [10]:
a = ytree.load("rockstar_halos/out_0.list")

In [11]:
print(a.derived_field_list)
print(a.field_list)

['halo_id', 'desc_id', 'mass', 'virial_mass', 'virial_radius', 'scale_radius', 'velocity_dispersion', 'position_x', 'position_y', 'position_z', 'velocity_x', 'velocity_y', 'velocity_z', 'angular_momentum_x', 'angular_momentum_y', 'angular_momentum_z', 'spin_parameter', 'T_|U|', 'redshift']
['ID', 'DescID', 'Mvir', 'Vmax', 'Vrms', 'Rvir', 'Rs', 'Np', 'X', 'Y', 'Z', 'VX', 'VY', 'VZ', 'JX', 'JY', 'JZ', 'Spin', 'rs_klypin', 'Mvir_all', 'M200b', 'M200c', 'M500c', 'M2500c', 'Xoff', 'Voff', 'spin_bullock', 'b_to_a', 'c_to_a', 'A[x]', 'A[y]', 'A[z]', 'b_to_a(500c)', 'c_to_a(500c)', 'A[x](500c)', 'A[y](500c)', 'A[z](500c)', 'T/|U|', 'M_pe_Behroozi', 'M_pe_Diemer', 'scale_factor', 'uid', 'desc_uid']


In [54]:
df_trees = {}
counter = 0
for tree in a:
    scale = tree["prog", "scale_factor"]
    assert(scale.size == np.unique(scale).size)
    Mvir = tree["prog", "Mvir_all"]
    Vrms = tree["prog", "Vrms"]
    Rvir = tree["prog", "Rvir"]
    Spin = tree["prog", "Spin"]
    Z = tree["prog", "redshift"]
    ti = str(counter)
    d = pd.DataFrame({('Mvir'): Mvir, ('Vrms'): Vrms, ('Rvir'): Rvir,('Spin'): Spin,('redshift'): Z})
    df_trees[ti] = d
    counter += 1

df = pd.concat(df_trees)
df2 = df.unstack(0)

In [56]:
redshift_dict = {}
for row in df2["redshift"].itertuples():
    idx = row[0]
    key = np.round(np.max(row[1]),2)
    redshift_dict[key] = {'Rvir':np.array(df2["Rvir"].loc[idx]), 'Mvir':np.array(df2["Mvir"].loc[idx]),
                          'Spin':np.array(df2["Spin"].loc[idx]),'Vrms':np.array(df2["Vrms"].loc[idx])}

In [74]:
np.warnings.filterwarnings('ignore')

sc_x = LinearScale()
sc_y = LinearScale()
sc_xax = Axis(label=('Vrms'), scale=sc_x, orientation='horizontal')
sc_yax = Axis(label=('Mvir'), scale=sc_y, orientation='vertical')
sc_x2 = LinearScale()
sc_y2 = LinearScale()
sc_xax2 = Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal')
sc_yax2 = Axis(label=('Mvir'), scale=sc_y2, orientation='vertical')

Z = 0.00
chosen_y_axis = redshift_dict[Z]["Mvir"]
x_y_cond = np.ones(len(Mvir),dtype=bool)

x_orig = redshift_dict[Z]["Vrms"]
y_orig = redshift_dict[Z]["Mvir"]

scatter = bqplot.Scatter(x = x_orig, y = y_orig, 
                        scales = {'x': sc_x, 'y': sc_y}, colors = ['blue'])
scatter2 = bqplot.Scatter(x = x_orig, y = y_orig,
                        scales = {'x': sc_x2, 'y': sc_y2}, colors = ['red'], visible = False)

db_scat_brush = HTML(value='[]')

# Y-axis selector widget
def dropdown_callback(change):
    highlight = bselect.selected
    Z = redshift_slider.value
    x_orig = redshift_dict[Z]["Vrms"]
    y_orig = redshift_dict[Z]["Mvir"]
    chosen_y_axis = redshift_dict[Z][change["new"]]
    scatter2.visible = False
    x_condition = (x_orig>=float(highlight[0][0])) & (x_orig<=float(highlight[1][0]))
    y_condition = (y_orig>=float(highlight[0][1])) & (y_orig<=float(highlight[1][1]))
    x_y_cond = x_condition & y_condition
    scatter2.x = x_orig[x_y_cond]
    scatter2.y = chosen_y_axis[x_y_cond]
    fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
               Axis(label=(change['new']), scale=sc_y2, orientation='vertical')]
    fig_scat_brush2.title = 'Vrms vs. ' + change['new']
    scatter2.visible = True

y_options = ['Rvir', 'Mvir','Spin']

y_selector = widgets.Dropdown(
    handler=dropdown_callback,
    options=y_options,
    value='Mvir',
    description='Y Axis:',
    )

y_selector.observe(dropdown_callback, names=['value'])

# Graph brush selector widget
def brush_callback(change):    
    highlight = bselect.selected
    Z = redshift_slider.value
    db_scat_brush.value = str(highlight)
    key = y_selector.value
    x_orig = redshift_dict[Z]["Vrms"]
    y_orig = redshift_dict[Z]["Mvir"]
    chosen_y_axis = redshift_dict[Z][key]
    if change.new is not None:
        try:
            scatter2.visible = False
            x_condition = (x_orig>=float(highlight[0][0])) & (x_orig<=float(highlight[1][0]))
            y_condition = (y_orig>=float(highlight[0][1])) & (y_orig<=float(highlight[1][1]))
            x_y_cond = x_condition & y_condition
            scatter2.x = x_orig[x_y_cond]
            scatter2.y = chosen_y_axis[x_y_cond]
            fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                       Axis(label=(key), scale=sc_y2, orientation='vertical')]
            scatter2.visible = True
        except IndexError:
            ;

z_values = np.array(list(redshift_dict.keys()))
# Redshift slider widget
redshift_slider = widgets.SelectionSlider(options=z_values,value=0.00,description='Redshift',
                                          disabled=False,continuous_update=False,
                                          orientation='horizontal',readout=True,
                                          layout=Layout(width='100%',position='top'))

def on_value_change(change):
    Z = change['new']
    try:
        scatter2.visible = False
        bselect.reset()
        x_orig = redshift_dict[Z]["Vrms"]
        y_orig = redshift_dict[Z]["Mvir"]
        scatter.x = x_orig
        scatter.y = y_orig
        fig_scat_brush1.axes =[Axis(label=('Vrms'), scale=sc_x, orientation='horizontal'),
                               Axis(label=('Mvir'), scale=sc_y, orientation='vertical')]
        key = y_selector.value
        chosen_y_axis = redshift_dict[Z][key]
        scatter2.x = x_orig
        scatter2.y = chosen_y_axis
        fig_scat_brush2.axes =[Axis(label=('Vrms'), scale=sc_x2, orientation='horizontal'),
                               Axis(label=(key), scale=sc_y2, orientation='vertical')]
    except KeyError:
        ;

redshift_slider.observe(on_value_change, names='value')

bselect = bqplot.interacts.BrushSelector(x_scale=sc_x, y_scale=sc_y, marks=[scatter],clear=False)

bselect.observe(brush_callback, names=['selected'])

fig_scat_brush1 = Figure(marks=[scatter], axes=[sc_xax, sc_yax], title='Vrms vs. Mvir',
                        interaction=bselect)
fig_scat_brush2 = Figure(marks=[scatter2], axes=[sc_xax2, sc_yax2], title='Vrms vs. Mvir')

left_box = VBox([db_scat_brush, fig_scat_brush1])
right_box = VBox([fig_scat_brush2, y_selector])
graph_box = HBox([left_box, right_box])

VBox([graph_box,redshift_slider])

VBox(children=(HBox(children=(VBox(children=(HTML(value='[]'), Figure(axes=[Axis(label='Vrms', scale=LinearScale()), Axis(label='Mvir', orientation='vertical', scale=LinearScale())], fig_margin={'top': 60, 'bottom': 60, 'left': 60, 'right': 60}, interaction=BrushSelector(marks=[Scatter(colors=['blue'], interactions={'hover': 'tooltip'}, scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'size': {'dimension': 'size'}, 'opacity': {'dimension': 'opacity'}, 'rotation': {'dimension': 'rotation'}, 'skew': {'dimension': 'skew'}}, tooltip_style={'opacity': 0.9}, x=array([  335.45999146,   335.36999512,   403.07998657,   457.76000977,
         285.27999878,   549.66998291,   429.67999268,   426.8999939 ,
         361.70001221,   143.02999878,   450.61999512,     0.        ,
         641.17999268,   564.36999512,   549.41998291,   166.91999817,
         564.07000732,   430.29000854,   550.53997803,   617.11999512,
         260.07000732,   604.88000488,   432.30999756,   464.30999756,
        1282.77001953,   130.63000488,   492.82998657,   603.94000244,
         503.13000488,   316.98999023,   578.02001953,   539.65002441,
         199.83000183,   570.78997803,   564.63000488,   353.1499939 ,
         535.7800293 ,   454.45001221,   500.73001099,   590.45001221,
         243.07000732,   503.76998901,   468.61999512,   215.1000061 ,
         438.29998779,   646.58001709,   280.5       ,   445.95001221,
         321.48999023,   568.73999023,   441.76998901,   558.28997803,
         442.51998901,   366.57998657,   165.8999939 ,   314.98999023,
         484.47000122,   227.82000732,   465.57998657,   297.5       ,
         529.65002441,   216.63999939,   448.64001465,   439.67001343,
         430.69000244,   376.3500061 ,   280.32000732,   491.10998535,
        1162.29003906,   485.45001221,   492.38000488,   330.75      ,
         346.60998535,   294.04998779,   422.48999023,   509.92001343,
         486.33999634,    58.38999939,   348.11999512,   441.38000488,
         455.01998901,   293.61999512,   435.97000122,   327.73001099,
         323.48001099,   334.07998657,   478.77999878,   388.29000854,
         422.19000244,   365.17999268,   158.53999329,   404.79998779,
         362.70001221,   141.46000671,   367.52999878,   465.6000061 ,
         389.1000061 ,   443.08999634,   437.47000122,   431.79998779,
         277.72000122,   390.5       ,   309.85998535,   421.10998535,
         411.98999023,   234.77000427,   202.72999573,   290.6499939 ,
         385.63000488,   376.92999268,   271.75      ,   359.8500061 ,
        1145.06005859,   764.47998047,   399.76000977,   249.88999939,
         423.5       ,   348.54000854,   314.5       ,   357.16000366,
         385.60998535,     0.        ,   304.86999512,   291.11999512,
        1156.84997559,   304.69000244,   295.60998535,   412.97000122,
         402.42001343,   410.73999023,    79.79000092,   160.02999878,
          57.20999908,   333.89001465,   277.07998657,   493.05999756,
         248.11999512,   345.1499939 ,   343.55999756,   236.57000732,
         326.85998535,   296.76000977,     0.        ,   350.45999146,
         260.3500061 ,   296.75      ,   222.91000366,   322.20999146,
         344.1000061 ,   321.22000122,   310.95001221,   196.58999634,
         308.30999756,   152.36999512,   347.52999878,   248.94000244,
         135.33000183,  1041.26000977,   299.04000854,   358.63000488,
         261.26000977,   207.32000732,   378.5       ,   282.70001221,
         219.83999634,   270.6000061 ,   186.91000366,   195.16999817,
         603.98999023,   171.44000244,   288.04000854,   370.67999268,
          97.62000275,   347.67999268,   235.97000122,   241.00999451,
         193.8500061 ,   403.44000244,   160.25      ,   202.52999878,
         332.07000732,   331.60998535,   329.14001465,   243.53999329,
         107.5   